In [4]:
import sys
import os
import numpy as np
from _3dpu_using_dfs import *
ROOT = "../"

In [5]:
import yaml

# Load the YAML file
with open(ROOT + 'paths.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access paths
data_path = config['paths']["data_big"]




In [23]:
import nibabel as nb
time = 3
data = nb.load(data_path).get_fdata()
data = np.array(data)[:100,:100,:40,time]

In [25]:
C = Resiuals(data)
C.map_nodes()
C.create_graph()
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.connex =[1]*len(C.mapping)
C.visited = [False]*len(C.mapping)
C.detect_cycles()


  0%|          | 14/193968 [00:00<02:15, 1430.46it/s]


KeyError: 127674

In [42]:
def dfs(C, v, path,path_dic,stack):
    C.visited[v] = True
    stack[v] = True
    path.append(v)
    path_dic[v] = len(path)-1

    for neighbour in C.Res_graph[v]:
        if not C.visited[neighbour]:
            if dfs(C,neighbour, path[:],path_dic,stack):
                return True
        elif stack[neighbour]:
            cycle_start_index = path_dic[neighbour]
            cycle = path[cycle_start_index:] + [neighbour]

            C.cycles.append(cycle)
            for point in cycle : 
                C.visited[point] = True
                C.incycles[point] = True
            return True

    stack[v] = False
    return False

def detect_cycles(C,stack):
    stack = [False]*len(C.mapping)
    for i in tqdm(range(len(C.mapping))):
        
        if not C.visited[i]:
            dfs(C,i, [],{},stack)
    

In [99]:
def iterative_dfs(C, start):
    stack = [(start, [start])]
    C.visited[start] = True
    path_dic = {start: 0}

    while stack:
        v, path, = stack.pop()
        C.visited[v] = True
        path_dic[v] = len(path) - 1

        for neighbour in C.Res_graph[v]:
            if not C.visited[neighbour]:
                newPath = path + [neighbour]
                stack.append((neighbour, newPath))
            elif neighbour in path:
                cycle_start_index = path_dic[neighbour]
                cycle = path[cycle_start_index:] + [neighbour]

                C.cycles.append(cycle)
                for point in cycle:
                    C.visited[point] = True
                    C.incycles[point] = True
                return True

    return stack

def detect_cycles_iterative(C):
    for i in tqdm(range(len(C.mapping))):
        if not C.visited[i]:
            iterative_dfs(C, i)
                

In [128]:
C.incycles = [False]*len(C.mapping)
C.visited =  [False]*len(C.mapping)
C.cycles = []
detect_cycles_iterative(C)


  0%|          | 0/193968 [00:00<?, ?it/s]

100%|██████████| 193968/193968 [00:05<00:00, 35104.19it/s]


In [6]:
Dict_of_rec = {}
for point in Points:
    Dict_of_rec[point] = []
for point in tqdm(Points):
    for i in range(len(C.cycles)):
        if point in C.cycles[i]:
            Dict_of_rec[point].append(i)


100%|██████████| 14824/14824 [00:15<00:00, 987.21it/s]


In [179]:
C.end_of_path = [False]*len(C.mapping)
for node in tqdm(range(len(C.mapping))):
    if C.Res_graph[node] == []:
        C.end_of_path[node] = True

def fill_open_paths(C):
    C.dict_open_paths = {}
    Not_in_cycles = []
    for node in range(len(C.mapping)):
        if not C.incycles[node] :
            Not_in_cycles.append(node)
            C.dict_open_paths[node] = 0
            
    print("detected")
    for node in tqdm(C.starting_open_paths):  
        cpt = 1
        # path = []
        while not C.incycles[node] and not C.end_of_path[node] :
            # path.append(node)
            C.dict_open_paths[node] = cpt
            node = C.Res_graph[node][0] 
        cpt = cpt + 1 # Get the first/next node in the path
        # C.open_paths.append(path)
        for point in Not_in_cycles:
            C.incycles[point] = (C.dict_open_paths[point] != 0)
    

  0%|          | 0/193968 [00:00<?, ?it/s]

100%|██████████| 193968/193968 [00:00<00:00, 1519717.03it/s]


In [201]:
def optimized_dfs(C):
    C.linear_paths = []
    visited = set()  # To keep track of visited nodes

    for start_node in tqdm(Begin):
        if start_node in visited or C.incycles[start_node]:
            continue

        node = start_node
        path = []
        while node not in visited:
            path.append(node)
            visited.add(node)
            
            if not node or node in C.end_of_path or C.incycles[node]:
                C.linear_paths.append(path)
                break

            # Assuming C.Res_graph[node] is a list of neighbors.
            # Modify this part as per your graph's structure.
            next_nodes = [n for n in C.Res_graph[node] if not C.incycles[n]]
            node = next_nodes[0] if next_nodes else None  # Proceed to the next node or stop if no more nodes

    return C.linear_paths


In [202]:
C.incycles = [False]*len(C.mapping)
C.open_paths = []
for path in C.cycles:
    for point in path:
        C.incycles[point] = True

optimized_dfs(C)

100%|██████████| 5466/5466 [01:26<00:00, 63.34it/s]


[[58298,
  193825,
  58144,
  193698,
  193699,
  193700,
  193701,
  58010,
  193593,
  193592,
  57885,
  193464,
  57744,
  193346,
  126426,
  193354,
  57752,
  193471,
  57751,
  193353,
  57632,
  193258,
  57492,
  57357,
  193027,
  57356,
  126276,
  193158,
  193159,
  57365,
  126170,
  57358,
  57493,
  193259,
  193260,
  126356,
  126352,
  None],
 [58453, 126767, 0],
 [59416, 1]]

In [210]:
for path in C.linear_paths:
    for point in path:
        if point:
            C.incycles[point] = True

In [215]:
leftovers = []
for point in range(len(C.mapping)):
    if C.incycles[point] == False:
        leftovers.append(point)

In [224]:
Begin = set(range(len(C.mapping)))
for point in tqdm(range(len(C.mapping))):
        Begin.difference_update(C.Res_graph[point])

100%|██████████| 193968/193968 [00:00<00:00, 1417837.38it/s]


In [226]:
my_set = {1, 2, 3, 4, 5, 6}
my_list = [2, 4, 9]

my_set.difference_update(my_list)
print(my_set)

{1, 3, 5, 6}


In [225]:
len(Begin)
len(C.)

2931

In [218]:
C.incycles[59415]

False

In [181]:
set(C.dict_open_paths.values())

{0, 1}

In [156]:
for op in C.open_paths:
    if 58329 in op:
        print(op)

[58329, 126712, 193867]
[58329, 126712, 193867]
[58329, 126712, 193867]


In [157]:
len(C.open_paths)/3

38.0

In [135]:
C.starting_open_paths[123]

63433

In [138]:
C.Res_graph[63433]

[63424, 4891]

In [139]:
for adjacency in C.Res_graph.values():
    if 4891 in adjacency :
        print(adjacency)

[63424, 4891]
[63424, 4891]


In [106]:
Not_in_cycles = []
for node in range(len(C.mapping)):
    if not C.incycles[node] :
        Not_in_cycles.append(node)
Not_in_cycles

[0,
 1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 164,
 165,
 168,
 169,
 170,
 174,
 175,
 176,
 179,
 180,
 181,
 182,
 183,
 185,
 189,
 190,
 192,
 193,
 194,
 196,
 197,
 198,
 199,
 200,
 201,
 203,
 204,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 216,
 217,
 218,
 221,
 222,
 223,
 224,

In [109]:
C.Res_graph[1336]

[129011]

In [110]:
C.incycles[129011]

False

In [39]:
J = [len(cycle) for cycle in C.cycles]

In [41]:
max(J)

89

In [65]:
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.visited = [1]*len(C.mapping)
C.dfs_iterative(613037)

In [79]:
cpt

58

In [62]:
for cycle in C.cycles:
    if 1334354 in cycle:
        print(len(cycle))

7416


In [50]:
for cycle in C.cycles:
    if 613037 in cycle:
        print(cycle)

[613018, 613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613018]
[613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613037]


In [13]:
C.visited

[]

In [16]:
len(np.where(np.array(C.incycles) == 1)[0])

606743

In [15]:
for open_path in C.open_paths:
    assert C.is_boundary(open_path[0]), "you hav error"

In [17]:
L = [len(C.Res_graph[node]) for node in C.starting_open_paths]
L

[1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,


In [24]:
incycles = [1]*len(C.mapping)
def detect_cycles(C):
    path_indices = {}
    for v in tqdm(range((len(C.mapping)))):
        if C.connex[v] != -1:
            dfs_iterative(C,v,path_indices)

def dfs_iterative(C, start,path_indices):
    stack = [(start, -1)]
    path_indices[start] = 0
    path = [start]
    while stack:
        node, parent = stack.pop()
        if C.connex[node] != -1:
            for neighbour in C.Res_graph[node]:
                if neighbour != parent:
                    if neighbour in path:
                        cycle_start_index = path.index(neighbour)
                        C.cycles.append(path[cycle_start_index:] + [neighbour])
                        for point in path[cycle_start_index:] + [neighbour]:
                            C.connex[point] = -1
                            C.incycles[point] = -1
                    else:
                        stack.append((neighbour, node))
                        path = path + [neighbour]
                        path_indices[neighbour] = len(path)
                    

            C.connex[node] = -1



In [25]:
C.cycles = []
for point in range(len(C.mapping)):
    C.connex[point] = 2

detect_cycles(C)

100%|██████████| 2023266/2023266 [00:31<00:00, 64708.27it/s]  


In [41]:
len(np.where(np.array(incycles) == 1)[0])/len(C.mapping)

0.3310459425503122

In [40]:
669794/len(C.mapping)

0.3310459425503122

In [38]:
L = [len(compo) for compo in C.connected_components.values()]
bad = L.index(1)
colour = list(C.connected_components.keys())[bad]
C.connected_components[colour]

[28]

In [39]:
C.Res_graph[28]

[1338522, 1338523]

In [ ]:
from tqdm import tqdm
for r in tqdm(C.list_res):
    C.Res_graph[R[r]] = []

  0%|          | 0/6069798 [00:00<?, ?it/s]

100%|██████████| 6069798/6069798 [00:09<00:00, 632648.60it/s] 


In [ ]:
R

[0]

In [ ]:
for res in tqdm(C.list_res):
    i,j,k,axe,value = res
    # if i not in [0,X-1] and j not in [0,Y-1] and k not in [0,Z-1]:
    C.nodes_of_not_boundary(res)

  0%|          | 0/2023266 [00:00<?, ?it/s]


KeyError: 0

In [ ]:
C.identify_connected_component()

In [ ]:
D = (C.connected_components).values()
Number = [len(k) for k in D]
R = sorted(Number)
R

[0]

In [ ]:
R

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
Number

[2,
 1781553,
 3,
 5,
 6,
 2,
 2,
 2,
 5,
 2,
 2,
 5,
 1,
 5,
 5,
 1,
 4,
 2,
 8,
 4,
 2,
 6,
 2,
 4,
 10,
 4,
 4,
 4,
 4,
 16,
 6,
 4,
 6,
 15,
 8,
 4,
 4,
 8,
 4,
 4,
 12,
 4,
 2,
 3,
 6,
 24,
 4,
 12,
 6,
 4,
 4,
 2,
 4,
 4,
 6,
 4,
 4,
 8,
 8,
 22,
 4,
 14,
 8,
 6,
 2,
 4,
 6,
 2,
 10,
 4,
 4,
 4,
 2,
 7,
 4,
 8,
 10,
 14,
 4,
 4,
 6,
 6,
 6,
 4,
 9,
 4,
 3,
 8,
 4,
 2,
 12,
 6,
 2,
 2,
 4,
 6,
 10,
 4,
 4,
 9,
 6,
 4,
 8,
 4,
 4,
 10,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 1,
 6,
 4,
 2,
 10,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 4,
 14,
 6,
 4,
 1,
 4,
 4,
 6,
 3,
 4,
 4,
 6,
 4,
 4,
 6,
 10,
 5,
 4,
 4,
 14,
 4,
 4,
 10,
 6,
 12,
 8,
 4,
 16,
 2,
 8,
 3,
 6,
 4,
 4,
 4,
 4,
 14,
 12,
 6,
 4,
 4,
 4,
 4,
 4,
 10,
 9,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 10,
 4,
 4,
 4,
 5,
 6,
 10,
 6,
 5,
 4,
 6,
 6,
 4,
 4,
 8,
 11,
 4,
 11,
 2,
 4,
 14,
 3,
 4,
 2,
 3,
 1,
 2,
 1,
 1,
 3,
 5,
 8,
 4,
 3,
 5,
 2,
 3,
 4,
 7,
 2,
 2,
 23,
 2,
 3,
 7,
 2,
 2,
 16,
 3,
 3,
 3,
 4,
 2,
 2,
 6,
 2,
 19,
 3

In [ ]:
0 in (set(list(C.connex.values())))

False

In [ ]:
R = []
for key in tqdm(C.Res_graph.keys()):
    if len(C.Res_graph[key]) not in [0,1,2,3]:
        R.append(key)

100%|██████████| 2023266/2023266 [00:00<00:00, 2402570.20it/s]
